In [46]:
import pandas as pd
from gurobipy import Model, GRB

In [47]:
# Load data
supply_data = pd.read_csv('/Users/mahinbindra/Downloads/ecogreen_energy_supply.csv')
demand_data = pd.read_csv('/Users/mahinbindra/Downloads/ecogreen_energy_demand.csv')

In [48]:
# Initialize the model
model = Model("EcoGreen Energy Expansion")

In [49]:
# Number of sites and provinces
num_sites = supply_data.shape[0]
num_provinces = demand_data.shape[0]

In [50]:
# Decision Variables
x = model.addVars(num_sites, vtype=GRB.BINARY, name="Open")
y = model.addVars(num_sites, num_provinces, vtype=GRB.CONTINUOUS, name="Energy_Transferred")

In [51]:
# Objective Function: Minimize total costs
model.setObjective(
    sum(supply_data.loc[i, 'Fixed'] * x[i] for i in range(num_sites)) +
    sum(supply_data.loc[i, f'Province {j+1}'] * y[i, j] for i in range(num_sites) for j in range(num_provinces)),
    GRB.MINIMIZE
)

In [52]:
# Constraints
# Capacity constraints
model.addConstrs((sum(y[i, j] for j in range(num_provinces)) <= supply_data.loc[i, 'Capacity'] * x[i]
                  for i in range(num_sites)), "Capacity")

# Demand constraints
model.addConstrs((sum(y[i, j] for i in range(num_sites)) >= demand_data.loc[j, 'Demand']
                  for j in range(num_provinces)), "Demand")

# Mutual Exclusivity and Dependency
model.addConstr(x[9] <= 1 - x[14], "Mutual_Exclusivity_10_15")
model.addConstr(x[9] <= 1 - x[19], "Mutual_Exclusivity_10_20")
model.addConstr(x[14] <= 1 - x[19], "Mutual_Exclusivity_15_20")
model.addConstr(2 * x[2] <= x[3] + x[4], "Dependency_3_4_5")
model.addConstr(x[4] <= x[7] + x[8], "Dependency_5_on_8_9")

# Regional Plant Limits
model.addConstr(sum(x[i] for i in range(10)) <= 2 * sum(x[i] for i in range(10, 20)), "Regional_Limits")

# Energy Output Mix
total_energy = sum(y[i, j] for i in range(5) for j in range(num_provinces))
total_all_energy = sum(y[i, j] for i in range(num_sites) for j in range(num_provinces))
model.addConstr(total_energy >= 0.3 * total_all_energy, "Minimum_Output")

# Provincial Energy Caps
model.addConstrs((y[i, j] <= 0.5 * demand_data.loc[j, 'Demand'] for i in range(num_sites) for j in range(num_provinces)),
                 "Max_Energy_Per_Province")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,


In [53]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 237 rows, 220 columns and 852 nonzeros
Model fingerprint: 0x5a7d6b2f
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 2.938222e+08
Presolve removed 202 rows and 0 columns
Presolve time: 0.00s
Presolved: 35 rows, 220 columns, 649 nonzeros
Variable types: 200 continuous, 20 integer (20 binary)

Root relaxation: objective 1.674269e+08, 71 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.6743e+08    0    4 2.9382e+08 1.6743e+08  43.

In [54]:
# Display results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(f"{v.varName} = {v.x}")
else:
    print("No optimal solution found.")

Optimal solution found:
Open[0] = 1.0
Open[1] = 1.0
Open[2] = 1.0
Open[3] = 1.0
Open[4] = 1.0
Open[5] = -0.0
Open[6] = 1.0
Open[7] = 1.0
Open[8] = -0.0
Open[9] = -0.0
Open[10] = 1.0
Open[11] = -0.0
Open[12] = -0.0
Open[13] = -0.0
Open[14] = 1.0
Open[15] = 1.0
Open[16] = 1.0
Open[17] = 0.0
Open[18] = -0.0
Open[19] = -0.0
Energy_Transferred[0,0] = 0.0
Energy_Transferred[0,1] = 0.0
Energy_Transferred[0,2] = 0.0
Energy_Transferred[0,3] = 81571.5
Energy_Transferred[0,4] = 0.0
Energy_Transferred[0,5] = 0.0
Energy_Transferred[0,6] = 0.0
Energy_Transferred[0,7] = 0.0
Energy_Transferred[0,8] = 0.0
Energy_Transferred[0,9] = 43688.5
Energy_Transferred[1,0] = 49395.5
Energy_Transferred[1,1] = 0.0
Energy_Transferred[1,2] = 0.0
Energy_Transferred[1,3] = 0.0
Energy_Transferred[1,4] = 0.0
Energy_Transferred[1,5] = 0.0
Energy_Transferred[1,6] = 0.0
Energy_Transferred[1,7] = 85696.5
Energy_Transferred[1,8] = 0.0
Energy_Transferred[1,9] = 0.0
Energy_Transferred[2,0] = 0.0
Energy_Transferred[2,1] = 0.0
En

In [55]:
#PART A

# Calculate the number of distinct provinces each plant can supply
distinct_provinces_per_plant = {i: sum(1 for j in range(num_provinces) if y[i, j].X > 0) for i in range(num_sites)}

# Print the results for each plant
for i in range(num_sites):
    print(f"Plant {i+1} can supply {distinct_provinces_per_plant[i]} distinct provinces.")


Plant 1 can supply 2 distinct provinces.
Plant 2 can supply 2 distinct provinces.
Plant 3 can supply 2 distinct provinces.
Plant 4 can supply 1 distinct provinces.
Plant 5 can supply 3 distinct provinces.
Plant 6 can supply 0 distinct provinces.
Plant 7 can supply 3 distinct provinces.
Plant 8 can supply 2 distinct provinces.
Plant 9 can supply 0 distinct provinces.
Plant 10 can supply 0 distinct provinces.
Plant 11 can supply 3 distinct provinces.
Plant 12 can supply 0 distinct provinces.
Plant 13 can supply 0 distinct provinces.
Plant 14 can supply 0 distinct provinces.
Plant 15 can supply 3 distinct provinces.
Plant 16 can supply 3 distinct provinces.
Plant 17 can supply 3 distinct provinces.
Plant 18 can supply 0 distinct provinces.
Plant 19 can supply 0 distinct provinces.
Plant 20 can supply 0 distinct provinces.


In [56]:
#PART B

num_variables = num_sites + (num_sites * num_provinces)
print(f"Total number of decision variables: {num_variables}")

Total number of decision variables: 220


In [57]:
# Constraints for each plant i
for i in range(num_sites):
    model.addConstr(sum(y[i, j] for j in range(num_provinces)) <= supply_data.loc[i, 'Capacity'] * x[i], f"Capacity_{i}")


In [58]:
optimal_cost = model.ObjVal
print(f"Optimal Cost: {optimal_cost}")

Optimal Cost: 193005688.7898808


In [59]:
num_plants_opened = sum(1 for i in range(num_sites) if x[i].X > 0.5)
print(f"Number of power plants established: {num_plants_opened}")

Number of power plants established: 11


In [60]:
plants_per_province = [sum(1 for i in range(num_sites) if y[i, j].X > 0) for j in range(num_provinces)]
highest = max(plants_per_province)
lowest = min(plants_per_province)
print(f"Highest number of plants per province: {highest}")
print(f"Lowest number of plants per province: {lowest}")

Highest number of plants per province: 4
Lowest number of plants per province: 2


In [61]:
model.setParam('PoolSearchMode', 2)  # Find n best solutions
model.setParam('PoolGap', 0.01)      # Within 1% of the optimal
model.optimize()

number_of_solutions = model.SolCount
print(f"Number of feasible solutions within 1% of the optimal: {number_of_solutions}")

Set parameter PoolSearchMode to value 2
Set parameter PoolGap to value 0.01
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 257 rows, 220 columns and 1072 nonzeros
Model fingerprint: 0x803f197a
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]

MIP start from previous solve produced solution with objective 1.93006e+08 (0.00s)
Loaded MIP start from previous solve with objective 1.93006e+08

Presolve removed 222 rows and 0 columns
Presolve time: 0.00s
Presolved: 35 rows, 220 columns, 649 nonzeros
Variable types: 200 continuous, 20 integer (20 binary)

Root relaxation: objective 1.674269e+08, 71 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    

In [62]:
# Remove the constraint related to the 50% energy supply cap
for i in range(num_sites):
    for j in range(num_provinces):
        constraint_name = f"Max_Energy_Per_Province_{i}_{j}"
        if model.getConstrByName(constraint_name):  # Check if the constraint exists
            model.remove(model.getConstrByName(constraint_name))

# Update the model to apply the changes
model.update()

# Optimize the model after removing the constraint
model.optimize()

# Display results and count the number of power plants established
if model.status == GRB.OPTIMAL:
    print("Optimal solution found after removing 50% cap constraint:")
    num_plants_opened = sum(1 for i in range(num_sites) if x[i].X > 0.5)
    print(f"Number of power plants established: {num_plants_opened}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 257 rows, 220 columns and 1072 nonzeros
Model fingerprint: 0x803f197a
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolved: 35 rows, 220 columns, 649 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 1
  Clique: 1
  MIR: 1

Explored 15 nodes (237 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 1.93006e+08 1.93021e+08 
No other solutions better than 1.93021e+08

Optimal solution found (tolerance 1.00e-04)
Best objective 1.930056887899e+08, best bound 1.930056887899e+08, gap 0.0000%
Optimal solution found after removing 50% cap cons

In [63]:
print("######################## PART A######################")

######################## PART A######################


In [64]:
def run_model(variable_type):
    # Load your data setup here
    supply_data = pd.read_csv('/Users/mahinbindra/Downloads/ecogreen_energy_supply.csv')
    demand_data = pd.read_csv('/Users/mahinbindra/Downloads/ecogreen_energy_demand.csv')
    num_sites = supply_data.shape[0]
    num_provinces = demand_data.shape[0]

# Run models with different variable types
continuous_obj_val, continuous_status = run_model(GRB.CONTINUOUS)
integer_obj_val, integer_status = run_model(GRB.INTEGER)

# Output the results
print(f"Continuous model objective value: {continuous_obj_val if continuous_status == GRB.OPTIMAL else 'No solution found'}")
print(f"Integer model objective value: {integer_obj_val if integer_status == GRB.OPTIMAL else 'No solution found'}")

TypeError: cannot unpack non-iterable NoneType object